In [ ]:
import pandas as pd
import numpy as np
import datetime
from sklearn.ensemble import ExtraTreesRegressor
from lightgbm import LGBMRegressor
from sklearn.ensemble import StackingRegressor

# New Section

In [ ]:
from fastai.imports import*

In [ ]:
train = pd.read_csv('/content/train.csv')
test = pd.read_csv('/content/test.csv')

In [ ]:
# using fast AI for creating variable
# other variable are manually created from data insights i.e. EDA
def add_datepart(df, fldname, drop=True):
    fld = df[fldname]
    if not np.issubdtype(fld.dtype, np.datetime64):
        df[fldname] = fld = pd.to_datetime(fld, infer_datetime_format=True)
    targ_pre = re.sub('[Dd]ate$', '', fldname)
    for n in ('Year', 'Month', 'Week', 'Day', 'Dayofweek', 'Dayofyear',
            'Is_month_end', 'Is_month_start', 'Is_quarter_end', 'Is_quarter_start', 'Is_year_end', 'Is_year_start'):
        df[targ_pre+n] = getattr(fld.dt,n.lower())
    df[targ_pre+'Elapsed'] = fld.astype(np.int64) // 10**9
    if drop: df.drop(fldname, axis=1, inplace=True)

In [ ]:
train.head()

,DateTime,Junction,Vehicles,ID
0,2015-11-01 00:00:00,1,15,20151101001
1,2015-11-01 01:00:00,1,13,20151101011
2,2015-11-01 02:00:00,1,10,20151101021
3,2015-11-01 03:00:00,1,7,20151101031
4,2015-11-01 04:00:00,1,9,20151101041


In [ ]:
add_datepart(train, 'DateTime',drop = True)

In [ ]:
add_datepart(test, 'DateTime',drop = True)

In [ ]:
train.head()

,Junction,Vehicles,ID,DateTimeYear,DateTimeMonth,DateTimeWeek,DateTimeDay,DateTimeDayofweek,DateTimeDayofyear,DateTimeIs_month_end,DateTimeIs_month_start,DateTimeIs_quarter_end,DateTimeIs_quarter_start,DateTimeIs_year_end,DateTimeIs_year_start,DateTimeElapsed
0,1,15,20151101001,2015,11,44,1,6,305,False,True,False,False,False,False,1446336000
1,1,13,20151101011,2015,11,44,1,6,305,False,True,False,False,False,False,1446339600
2,1,10,20151101021,2015,11,44,1,6,305,False,True,False,False,False,False,1446343200
3,1,7,20151101031,2015,11,44,1,6,305,False,True,False,False,False,False,1446346800
4,1,9,20151101041,2015,11,44,1,6,305,False,True,False,False,False,False,1446350400


In [ ]:
# four junctions are there hence individual prediction are given for each of the junction
junction = train['Junction'].unique()
junction

array([1, 2, 3, 4])

In [ ]:
# used stack model for prediction at each juction with the help of LGBM, and Extra tree
# hyper-para-tuning of each model is done separately
# then best paramters where selected for the stack model 
# once the output of both the model are obtained Linear regression is used to reduce error 
# output of both the model were used to pred the final output using stack modelling 
# here param are not mentioned 
results = pd.DataFrame()
for item in junction:
    X = train[train['Junction'] == item]
    
    Y = test[test['Junction'] == item]
    
    X_train = X.drop(columns = 'Junction', axis = 1)
    
    X_train.set_index('ID', inplace = True)
    
    x_train = X_train.drop(columns = 'Vehicles')
    y_train = X_train['Vehicles']
    
    Y_test = Y.drop(columns = 'Junction', axis = 1)
    
    Y_test.set_index('ID', inplace = True)
    
    print(x_train.shape)
    estimator = [
                 ('EXT', ExtraTreesRegressor(random_state= 42, bootstrap= True)),
                 ('LGBM',LGBMRegressor())
                ]
    stack_model = StackingRegressor(estimators= = estimator, final_estimator= LinearRegression())
    r = stack_model.predict(Y_test)
    q = pd.DataFrame(r)
    results = results.append(q)
    print(x_train.shape)
    print(Y_test.shape)

(2952, 13)
(2952, 13)
(2952, 13)
(2952, 13)


In [ ]:
# sanity and final edit of results were done in excel itself
# along with this armia is used for the predection and the weighted average of arima and stack model is submitted
# weighted average is taken since for two of jucntion arima is performing better and for two junction stack hence weighted average is used 
# 60%arima and 40% stack for predicting final forecasted value 
# public dataset rank 2
# final private dataset rank 4 
 results